In [1]:
!nvidia-smi

Thu Jun 15 20:17:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:09:00.0 Off |                  N/A |
| 66%   72C    P2   321W / 350W |  24013MiB / 24576MiB |     90%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
|  0%   

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from train_utils import TrainWrap
import pandas as pd


model_ckpt = 'roberta-base'

tw = TrainWrap(model_ckpt)


/home/14cfl/anaconda3/envs/pt2/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [10]:
import pickle as pkl


with open('data/prompt-ood.pkl', 'rb') as f:
    data = pkl.load(f)
    


In [11]:
data

{'train': [{
    "guid": "25360",
    "label": 2,
    "meta": {
      "definition": "bias-motivated, hostile and malicious language targeted at a person/group because of their actual or perceived innate characteristics, especially when the group is unnecessarily labeled",
      "target": "hate speech"
    },
    "text_a": "This Florida sun gone make me black as fuck \ud83d\ude2d\ud83d\ude2d",
    "text_b": "",
    "tgt_text": null
  },
  {
    "guid": "20056",
    "label": 2,
    "meta": {
      "definition": "bias-motivated, hostile and malicious language targeted at a person/group because of their actual or perceived innate characteristics, especially when the group is unnecessarily labeled",
      "target": "hate speech"
    },
    "text_a": "Trans culture is seeing that and knowing that you look amazing, but having dysphoria leave you feeling hollow and constantly on the verge of tears. URL",
    "text_b": "",
    "tgt_text": null
  },
  {
    "guid": "42157",
    "label": 2,
    "

In [14]:
defs = pd.read_csv('data/definitions.csv')

defs = defs.append({'promptName': 'hate speech', 'name': 'hate_speech_score_sigmoid', 
                    'definition': 'bias-motivated, hostile and malicious language targeted at a person/group because of their actual or perceived innate characteristics, especially when the group is unnecessarily labeled.'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Implicit Hatred', 'name': 'other', 
                    'definition': 'the use of coded or indirect language such as sarcasm, metaphor and circumlocution to disparage a protected group or individual, or to convey prejudicial and harmful views about them.'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Hate Speech', 'name': 'labels', 
                    'definition': 'bias-motivated, hostile and malicious language targeted at a person/group because of their actual or perceived innate characteristics, especially when the group is unnecessarily labeled.'
                   }, ignore_index=True)


defs = defs.append({'promptName': 'Dehumanization', 'name': 'dehumanize', 
                    'definition': 'the process of depriving a person or group of positive human qualities.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Violent language', 'name': 'violence', 
                    'definition': 'calling for the unlawful exercise of physical force or intimidation by the exhibition of such force on a large group of people.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Genocide', 'name': 'genocide', 
                    'definition': 'calling for the deliberate killing of a large group of people.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Respect', 'name': 'respect', 
                    'definition': 'a feeling of deep admiration for someone or something elicited by their abilities, qualities, or achievements.'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Insult', 'name': 'insult', 
                    'definition': 'to speak to or treat with disrespect or scornful abuse.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Humiliation', 'name': 'humiliate', 
                    'definition': 'making (someone) feel ashamed and foolish by injuring their dignity and self-respect, especially publicly.'
                   }, ignore_index=True)

defs = defs.append({'promptName': 'Superiority', 'name': 'status', 
                    'definition': 'higher in rank, status, or quality.'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Attack', 'name': 'attack_defend', 
                    'definition': 'an aggressive and violent action against a person or place, and defense is defined as speak or write in favor of (an action or person); attempt to justify.'
                   }, ignore_index=True)

/tmp/ipykernel_387685/3357330585.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'hate speech', 'name': 'hate_speech_score_sigmoid',
/tmp/ipykernel_387685/3357330585.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'Implicit Hatred', 'name': 'other',
/tmp/ipykernel_387685/3357330585.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'Hate Speech', 'name': 'labels',
/tmp/ipykernel_387685/3357330585.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'Dehumanization', 'name': 'dehumanize',
/tmp/i

In [15]:
train_implicit = pd.read_csv('data/train_full.tsv', delimiter='\t')
train_sbic = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.trn.csv")

val_implicit = pd.read_csv('data/validation_full.tsv', delimiter='\t')
val_sbic = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.dev.csv")


train_measuring = pd.read_csv('data/train_measuring.csv')
val_measuring = pd.read_csv('data/val_measuring.csv')

In [16]:
train_implicit

,Unnamed: 0,text,implicit_class,extra_implicit_class,target,implied_statement,prev_label,label,id,post
0,20996,"Yes ma'am, and we'll warm up @TheRealDonaldTru...",NaN,NaN,NaN,NaN,explicit_hate,False,implicit_20976,"Yes ma'am, and we'll warm up @TheRealDonaldTru..."
1,21565,REMINDER: femoids would rather fuck ABUSIVE CH...,NaN,NaN,NaN,NaN,implicit_hate,False,implicit_21545,REMINDER: femoids would rather fuck ABUSIVE CH...
2,14513,. once argued partial-birth abortions nec . fo...,NaN,NaN,NaN,NaN,not_hate,False,implicit_14499,. once argued partial-birth abortions nec . fo...
3,7314,does this pick upset you ? good racist dolts,NaN,NaN,NaN,NaN,not_hate,False,implicit_7306,does this pick upset you ? good racist dolts
4,10547,that's not true . whites don't scream their ci...,NaN,NaN,NaN,NaN,not_hate,False,implicit_10534,that's not true . whites don't scream their ci...
...,...,...,...,...,...,...,...,...,...,...
17235,8579,"rt "" why would ( ( ( they ) ) ) want both pa...",NaN,NaN,NaN,NaN,implicit_hate,False,implicit_8570,"rt "" why would ( ( ( they ) ) ) want both pa..."
17236,3256,sadly this is what the older jewish sect wants...,stereotypical,NaN,jewish people,jewish people want a race war.,implicit_hate,True,implicit_3253,sadly this is what the older jewish sect wants...
17237,15405,i'm german ethnic group my roots is anglo saxo...,NaN,NaN,NaN,NaN,not_hate,False,implicit_15390,i'm german ethnic group my roots is anglo saxo...
17238,21543,"Also, I can't imagine anything more counterpro...",NaN,NaN,NaN,NaN,explicit_hate,False,implicit_21523,"Also, I can't imagine anything more counterpro..."


In [18]:
# implicit_labels = ['white_grievance','incitement', 'inferiority', 'stereotypical', 'irony', 'threatening', 'other']
implicit_labels = ['implicit_class']
sbic_labels = ['offensiveYN', 'intentYN', 'whoTarget']

_, train_sbic_enc = tw.preprocess_prompt(train_sbic, t='continuous', target=sbic_labels, defs=defs, include_def=False, id_col='Unnamed: 0')
_, train_implicit_enc = tw.preprocess_prompt(train_implicit, t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')
_, val_sbic_enc = tw.preprocess_prompt(val_sbic, t='continuous', target=sbic_labels, defs=defs, include_def=False, id_col='Unnamed: 0')
_, val_implicit_enc = tw.preprocess_prompt(val_implicit, t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')


100%|█████████████████████████████████████████████████████████| 35424/35424 [00:04<00:00, 8796.71it/s]


IndexError: single positional indexer is out-of-bounds

In [21]:
from collections import Counter
# _, train_implicit_enc = tw.preprocess_prompt(train_implicit, t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')
target_categories = list(set([x for y in train_sbic['targetCategory'] for x in literal_eval(y)]))

Counter([x for y in train_sbic['targetCategory'] for x in literal_eval(y)])

Counter({'social': 642,
         'culture': 2471,
         'gender': 3535,
         'disabled': 753,
         'race': 3883,
         'victim': 2027,
         'body': 441})

In [10]:
from ast import literal_eval
import html
import re



def process_alpaca(text, label, instruction=None):
#     instruction = defs
    
    out = {'instruction': [],
           'input': [],
           'output': []
          
          }
    
    for i in range(len(text)):
        
        outp = label[i]
            
        inp = text[i].strip()
        inp = re.sub(r'(u/.*? |@.*?( |$))', "<user> ", inp)
        inp = re.sub(r'\u2019', "'", inp)
        inp = re.sub(r'[“”]', '"', inp)
        inp = re.sub(r'\\u[a-f0-9]{4}', "'", inp)
        inp = re.sub(r'\(https://.*\)', "(URL)", inp)

        inp = re.sub(r'[\s]{2,}', " ", inp)
        inp = html.unescape(inp).encode('ascii', errors='ignore').decode()
        
        if type(instruction) is str:
            out['instruction'].append(instruction)
        elif type(instruction) is list:
            out['instruction'].append(instruction[i])
            
        out['input'].append(inp)
        out['output'].append(outp)
    return out

In [22]:

def openprompt_to_prompt(test_enc):
    instructions = []
    test_prompt = []
    
    output = []
    
    for i in range(len(test_enc)):
        ins = '{target} is defined as {target_def}'.format(target=test_enc[i].meta['target'],
                                                                         target_def=test_enc[i].meta['definition'])
        if test_enc[i].meta['target_groups'] is not None:
            ins += 'Target groups include {0}'.format(', '.join(test_enc[i].meta['target_groups']))
        instructions.append(ins)
        test_prompt.append('Post:{text} Q: Is this post a violation of {target}? Please answer Yes, No, or Unclear. A: '.format(
            target=test_enc[i].meta['target'], 
            target_def=test_enc[i].meta['definition'],
            text=test_enc[i].text_a))
        
        output.append(('Yes' if test_enc[i].label == 2 else ('Unclear' if test_enc[i].label == 1 else 'No')))
    return process_alpaca(test_prompt, output, instructions)

In [23]:
train_sbic_enc[-1], train_sbic.iloc[-1]

({
   "guid": "35503",
   "label": 2,
   "meta": {
     "definition": "targeting a person by implicating a group they belong to. Group implications are distinguished from individual-only attacks or insults that do not invoke power dynamics between groups.",
     "target": "Group implications",
     "target_groups": [
       "culture"
     ]
   },
   "text_a": "\ud83d\udea8BREAKING: illegal alien 5x deported on 7 felonies found not guilty in the murder of Kate Steinle who he shot with a gun\n\nJury full of communists just allowed a foreigner to execute an American citizen without repercussion\n\nThis is a glimpse into the future if we don\u2019t take drastic measures",
   "text_b": "",
   "tgt_text": null
 },
 Unnamed: 0                                                          35503
 post                    🚨BREAKING: illegal alien 5x deported on 7 felo...
 targetMinority                                             ["immigrants"]
 targetCategory                                          

In [24]:
Counter([x.label for x in train_sbic_enc])

Counter({2: 36678, 0: 38587, 1: 31007})

In [25]:
train_sbic_prompt = openprompt_to_prompt(train_sbic_enc)

In [26]:
from datasets import Dataset

train_sbic_prompt = Dataset.from_dict(train_sbic_prompt)

In [27]:
train_implicit_prompt = Dataset.from_dict(openprompt_to_prompt(train_implicit_enc))

In [28]:
train_implicit_prompt[0]

{'instruction': 'White Grievance is defined as frustration over a minority group’s perceived privilege and casting majority groups as the real victims of racism. This language is linked to extremist behavior and support for violence.',
 'input': "Post:ur right it's simple! islam is not part of our civilization via bill warner, phd: Q: Is this post a violation of White Grievance? Please answer Yes, No, or Unclear. A:",
 'output': 'No'}

In [30]:
from datasets import interleave_datasets

interleave_datasets([train_sbic_prompt, train_implicit_prompt]).to_json('crosstask.json')

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

27059672

In [31]:
train_sbic_prompt.to_json('train_sbic.json')


Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

46785919

In [32]:
train_implicit_prompt.to_json('train_implicit.json')

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

14227783

In [ ]:
instruction = '{target} is defined as {target_def}. Is this post a violation of any of the above definitions? Please answer Yes/No, and which policies. If there is no violations, output N/A.'
        